In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Obesity/GSE176025'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Small noncoding RNA expression data from type-2-diabetic and non-diabetic obese human liver biopsies"
!Series_summary	"Although the global prevalence of type 2 diabetes is still rising, the molecular mechanisms of dysregulated glucose and lipid metabolism are still incompletely understood. Epigenetic mechanisms were shown to have an impact in disease manifestation, but still the hepatic miRNA expression signature in diabetic subjects was not completely elucidated."
!Series_summary	"We used microarrays to performe a genome wide screen of the complete noncoding RNA transcriptome in order to identify dsyregulated miRNAs which target genes of the hepatic glucose and lipid metabolism."
!Series_overall_design	"Total RNA extracted from liver biopsies of 20 non-diabetic and 20 type-2-dabetic obese human subjects, matched for age and gender, were used for microarray measurement in order to establish a database for the human small noncoding RNA expression p

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if the dataset contains gene expression data
summary_info = "We used microarrays to performe a genome wide screen of the complete noncoding RNA transcriptome in order to identify dsyregulated miRNAs..."
if "RNA" in summary_info and "miRNA" not in summary_info and "noncoding" not in summary_info:
    is_gene_available = True

# Get data availability for Obesity, age, and gender
sample_characteristics = {
    0: ['tissue: liver'],
    1: ['condition: type-2-diabetic', 'condition: non-diabetic'],
    2: ['age: 50', 'age: 61', 'age: 54', 'age: 47', 'age: 51', 'age: 34', 'age: 53', 'age: 41', 'age: 48', 'age: 65', 
        'age: 49', 'age: 58', 'age: 42', 'age: 45', 'age: 62', 'age: 46', 'age: 60', 'age: 40', 'age: 36', 'age: 57', 
        'age: 33', 'age: 38', 'age: 43', 'age: 37', 'age: 55', 'age: 56', 'age: 52', 'age: 31'],
    3: ['bmi: 43.44', 'bmi: 46.92', 'bmi: 52.44', 'bmi: 38.34', 'bmi: 57.23', 'bmi: 52.01', 'bmi: 42.78', 'bmi: 62.28', 
        'bmi: 42.47', 'bmi: 57.16', 'bmi: 45.92', 'bmi: 50.78', 'bmi: 35', 'bmi: 47.4', 'bmi: 45', 'bmi: 50.2', 'bmi: 74.84', 
        'bmi: 60.45', 'bmi: 54', 'bmi: 52.58', 'bmi: 62.09', 'bmi: 45.94', 'bmi: 71.73', 'bmi: 58.11', 'bmi: 71.76', 'bmi: 57.01', 
        'bmi: 56.93', 'bmi: 56.69', 'bmi: 41.43', 'bmi: 68.68'],
    4: ['gender: m', 'gender: w']
}

# Set trait_row for Obesity, choose continuous data type since BMI values are not binary
trait_row = 3
def convert_trait(value):
    try:
        return float(value.split(": ")[1])
    except (ValueError, IndexError):
        return None

# Set age_row for age, choose continuous data type
age_row = 2
def convert_age(value):
    try:
        return float(value.split(": ")[1])
    except (ValueError, IndexError):
        return None

# Set gender_row for gender, choose binary data type
gender_row = 4
def convert_gender(value):
    gender_map = {"m": 1, "w": 0}
    try:
        return gender_map[value.split(": ")[1]]
    except (KeyError, IndexError):
        return None

save_cohort_info('GSE176025', './preprocessed/Obesity/cohort_info.json', is_gene_available, trait_row is not None)

if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Obesity', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Obesity/trait_data/GSE176025.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


A new JSON file was created at: ./preprocessed/Obesity/cohort_info.json
{'GSM5353151': [43.44, 50.0, 1.0], 'GSM5353152': [46.92, 61.0, 1.0], 'GSM5353153': [52.44, 54.0, 0.0], 'GSM5353154': [38.34, 47.0, 1.0], 'GSM5353155': [57.23, 47.0, 0.0], 'GSM5353156': [52.01, 51.0, 1.0], 'GSM5353157': [42.78, 34.0, 1.0], 'GSM5353158': [62.28, 47.0, 0.0], 'GSM5353159': [42.47, 53.0, 0.0], 'GSM5353160': [57.16, 41.0, 0.0], 'GSM5353161': [45.92, 48.0, 1.0], 'GSM5353162': [50.78, 65.0, 0.0], 'GSM5353163': [35.0, 49.0, 1.0], 'GSM5353164': [47.4, 58.0, 0.0], 'GSM5353165': [45.0, 34.0, 1.0], 'GSM5353166': [50.2, 42.0, 1.0], 'GSM5353167': [74.84, 45.0, 0.0], 'GSM5353168': [60.45, 61.0, 0.0], 'GSM5353169': [54.0, 62.0, 1.0], 'GSM5353170': [52.58, 46.0, 0.0], 'GSM5353171': [52.01, 34.0, 1.0], 'GSM5353172': [62.09, 60.0, 1.0], 'GSM5353173': [45.94, 40.0, 0.0], 'GSM5353174': [71.73, 54.0, 1.0], 'GSM5353175': [58.11, 62.0, 0.0], 'GSM5353176': [71.76, 36.0, 0.0], 'GSM5353177': [57.01, 57.0, 1.0], 'GSM5353178': 